### Use case for NFDInspector
In this use case, we will explore the capabilities of NFDInspector a little further. We will go through a typical application. In our scenario, a collection of stereo glass plates was digitized as part of a digitization project. The associated metadata was partly generated from existing legacy data and partly created by new indexing.
NDFInspector for LIDO-xml offers the possibility to adapt the parameters of the analysis to the respective use case. For this purpose, we will take a closer look at the configuration file and the setting options within it.<br>
The LIDO xml files were created and collected in the /LIDO_xml folder. There are a total of 2,063 LIDO-records in separate xml files. The existing data was generated from the portal __[westfalen.museum-digital.de](https://westfalen.museum-digital.de/objects?s=collection:670)__. For more information on the specifications of the LIDO format, please refer to the __[documentation](https://cidoc.mini.icom.museum/working-groups/lido/lido-overview/about-lido/what-is-lido/)__.

### Initialization
To begin the inspection, the NFDInspector package is imported and the lido_inspector is initialized. English is selected as the output language.

In [3]:
from nfdinspector.lido_inspector import LIDOInspector

lido_inspector = LIDOInspector(error_lang='en')

### Configuration
At this point, we will take a closer look at the configuration and therefore the configuration file. As there is no ready-made file available, the standard configuration is read from the ``LIDOInspector``. The dictionary created in this way is saved as a JSON file and can also be displayed in the notebook.

In [9]:
import json

lido_config = lido_inspector.configuration
with open('lido_config.json', "w") as outfile:
    json.dump(lido_config, outfile, indent=4)

lido_config

{'workID': {'pattern': ''},
 'title': {'inspect': True,
  'unique': True,
  'distinct_from_type': True,
  'min_word_num': 2,
  'max_word_num': 20},
 'category': {'inspect': True,
  'ref': True,
  'patterns': {'label': '', 'ref': ''}},
 'object_work_type': {'inspect': True,
  'ref': True,
  'patterns': {'label': '', 'ref': ''}},
 'classification': {'inspect': True,
  'ref': True,
  'patterns': {'label': '', 'ref': ''}},
 'object_description': {'inspect': True,
  'unique': True,
  'min_word_num': 20,
  'max_word_num': 500},
 'materials_tech': {'inspect': True, 'ref': True, 'differentiated': False},
 'object_measurements': {'inspect': True},
 'event': {'inspect': True, 'ref': True},
 'subject_concept': {'inspect': True, 'ref': True, 'min_num': 3},
 'resource': {'inspect': True},
 'record_type': {'inspect': True,
  'ref': True,
  'patterns': {'label': '', 'ref': ''}},
 'repository_name': {'inspect': True, 'ref': True},
 'record_source': {'inspect': True, 'ref': True},
 'record_rights': {'i

When inspecting the configuration file, you can see the various setting options of the NFDInspector. In addition to the option to check whether certain data fields have entries, properties such as uniqueness within a data set, minimum and maximum length of entries, presence of referencing or ID can be checked. It is also possible to check entries for formal correctness using regular expressions.<br>
In this case, the metadata should be checked for compatibility with the requirements of the German Digital Library. These requirements are listed on the __[DDB website](https://wiki.deutsche-digitale-bibliothek.de/display/DFD/Anforderungen+an+die+Lieferdaten)__ and can be viewed there. The minimum requirements stipulate that eight metadata elements must be present:

- Data record identifier
    - the identifier has to be stable
    - the identifier must not contain any spaces
- Data partner Identifiere
    - unique and persistent identifier for institution supplying the dataset to the DDB
    - The identifier should preferably be an International Standard Identifier for Libraries and Related Organizations (ISIL), ISO 15511, assigned by the German ISIL Agency and Sigelstelle at the Staatsbibliothek zu Berlin. For museums, the ISIL identifiers are assigned by the Institute for Museum Research Berlin.
- Link to the digital object
- Legal status of the digital object
- Object title
    - The object title should not be longer than 200 characters
    - It is not sufficient to enter the object type or the object name as the object title for the DDB
- Object type
    - The Object type has to be refrenced by a controlled vocabulary
- Media type

<br>
A configuration file can map the requirements. To do this, we modify the standard configuration as follows. The already exported file can either be edited and read in again, or the configuration within the LIDOInspector instance can be changed.